# Imports

In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 255)
from bs4 import BeautifulSoup
import nltk
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from pathlib import Path
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ThibJacq\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
nltk.download('stopwords', quiet=True)

True

`esa_csvs_raw.txt` is a tab-separated file of the raw data from the MySQL database. You will need to create this file if you want to include new data. The file contains the name of the CSV file as well as the contents of the CSV file (of the table). You need to make it tab-separated because MySQL will not separate things properly if you use a comma because the CSV contents include commas. If you *really* want to use a comma, there is a way to do it where there will be quotes added to separate the fields, but you will need to look up how to do it.

In [9]:
ROOT_PATH = Path('.').resolve().parents[1]

In [11]:
df_raw = pd.read_csv(str(ROOT_PATH / 'data/raw/csvs/esa_csvs_raw.txt'), sep='\t')

In [12]:
df_raw.iloc[0]

csvFileName                                                                                                                                                                                                                                        1059614_14_lattice-v_1.csv
csvText        [["Survey Date", "Aboriginal Communities", "Detail"], ["July 9 to 12, 2011", "Blueberry River First Nation <s>•</s>Halfway River First Nation <s>•</s>McLeod Lake Indian Band <s>•</s>North East Métis Association <s>•</s>", "Summer 2011 open water aquat...
Name: 0, dtype: object

In [13]:
df_raw['csvText'] = df_raw['csvText'].str.slice(0, 30_000) # limit to 30k chars since Excel can't handle more than 32k

In [14]:
assert (df_raw['csvText'].str.len()).max() <= 30_000 # sanity check

In [15]:
df_index = pd.read_csv(str(ROOT_PATH / 'data/raw/index_for_projects/ESA_website_ENG_2021_03_04_final_utf_8.csv'), index_col=0, encoding='utf-8')

In [16]:
df_index.rename(columns={"Old CSV Filename": "csvFileName"}, inplace=True)

In [20]:
df_index.head(2)

,Index,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,bad_csv,CSV Filename,csvFileName
0,9134,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABORIGINAL FIELD STUDY PARTICIPATION FOR THE PROJECT,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2.csv,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2.csv,1059614_14_lattice-v_1.csv
1,9135,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE (AITKEN CREEK SECTION),Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2.csv,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2.csv,1059614_17_lattice-v_1.csv


In [21]:
df_index.iloc[133:135] # sanity check

,Index,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,bad_csv,CSV Filename,csvFileName
133,9267,Table 2-1 Visual Sensitivity Ratings along Project RoW,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-25 ESA_Appendix_L_Part3of3 (A3Q6I1),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6I1,1059815,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059815,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix L: Socio-economic Technical Data Report,24.0,"Human, Technology, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-2-1-visual-sensitivity-ratings-along-project_pt-1_pg-8_doc-num-A3Q6I1.csv,8,22.0,2.77,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-2-1-visual-sensitivity-ratings-along-project_pt-1_pg-8_doc-num-A3Q6I1.csv,1059815_8_lattice-v_1.csv
134,9268,Table 6-1 Valued Components and Key Indicators – Soils,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-2 Volume 4 - ESA_Main_Part_1_of_5 (A3Q6F8),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6F8,1059854,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059854,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,"Table of Contents, Abbreviations, Section 1: Introduction, Section 2: Project Description, Section 3: Consultation and Engagement, Section 4: Route and Facility Site Selection, Section 5: Assessment Methods and Environmental Setting, Section 6: Soils,...",1.0,"Land, Water, Vegetation, Traditional Knowledge, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-6-1-valued-components-and-key-indicators--so_pt-1_pg-107_doc-num-A3Q6F8.csv,107,199.0,5.11,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-6-1-valued-components-and-key-indicators--so_pt-1_pg-107_doc-num-A3Q6F8.csv,1059854_107_lattice-v_1.csv


In [22]:
df_merged = df_raw[df_raw['csvFileName'].isin(list(df_index['csvFileName']))].reset_index(drop=True)

In [23]:
df_joined = pd.merge(df_index, df_merged, how='left', on='csvFileName')

In [25]:
df_joined.head(2)

,Index,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,bad_csv,CSV Filename,csvFileName,csvText
0,9134,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABORIGINAL FIELD STUDY PARTICIPATION FOR THE PROJECT,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2.csv,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2.csv,1059614_14_lattice-v_1.csv,"[[""Survey Date"", ""Aboriginal Communities"", ""Detail""], [""July 9 to 12, 2011"", ""Blueberry River First Nation <s>•</s>Halfway River First Nation <s>•</s>McLeod Lake Indian Band <s>•</s>North East Métis Association <s>•</s>"", ""Summer 2011 open water aquat..."
1,9135,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE (AITKEN CREEK SECTION),Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2.csv,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2.csv,1059614_17_lattice-v_1.csv,"[[""Field Site No."", ""Name, Legal Location"", ""UTM Coordinates (NAD83, Zone 10)"", ""Latitude/Longitude (DD-MM-SS), Elevation (m asl)"", ""Fish Stream Classification <s>1,7 </s>"", ""Fish and Wildlife Timing Window – Low Risk <s>2,7 </s>"", ""Riparian Managem..."


In [26]:
len(df_joined) # 28891

28891

In [27]:
del df_raw
del df_index
del df_merged

## Adding New Projects to the Index File

In [28]:
df_joined.rename(columns={"csvText": "text"}, inplace=True)
df_joined['label'] = -1

In [29]:
df_joined["text"] = df_joined["Title"] + " " + df_joined["text"]

In [30]:
df_joined['text'] = df_joined['text'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text(separator=' ')) # remove html tags
df_joined['text'] = df_joined['text'].replace('[^a-zA-Z0-9 ]', ' ', regex=True) # remove all non-alpha-numeric characters
df_joined['text'] = df_joined['text'].replace('\w{25,}', ' ', regex=True)
df_joined['text'] = df_joined['text'].replace('cid\d+', ' ', regex=True)
df_joined['text'] = df_joined['text'].replace(' s ', ' ', regex=True)
df_joined['text'] = df_joined['text'].replace(' +', ' ', regex=True) # remove all extra spaces in text

In [32]:
df_joined.text[103]

'Table 4 28 Assessed Value of an Average Single Family Home 2011 to 2013 Community 2011 2012 2013 2011 to 2013 increase Dawson Creek 212 870 229 000 238 000 12 Fort St John 274 035 305 000 326 000 19 Chetwynd 156 170 218 000 229 000 47 Hudson Hope 132 270 161 000 175 000 32 Taylor 208 098 247 000 281 000 35 SOURCE City of Dawson Creek 2013 '

In [33]:
df_joined.to_csv(str(ROOT_PATH / 'data/interim/Intermediate_Index_Files/esa_index_with_table_text_no_labels.csv'), index=False, encoding='utf-8')

In [34]:
from sklearn.model_selection import train_test_split
train_df, df_for_labeling = train_test_split(df_joined, test_size=1000, random_state=42, shuffle=True)

In [35]:
print(len(train_df))
print(len(df_for_labeling))

27891
1000


In [36]:
train_df.head(2)

,Index,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,bad_csv,CSV Filename,csvFileName,text,label
25896,35030,Table 2-6 Estimated Barge Traffic for Marine Terminal Construction,Table,Application for the Enbridge Northern Gateway Pipeline Project,Northern Gateway,5/27/2010,"Northern Gateway Pipelines Inc., as General Partner of Northern Gateway Pipelines Limited Partnership",Oil,B3-1 - Vol 6A P1 - Pipelines and Tank Terminal ESA (Part 1 of 5) - A1T0F1,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/620129,A1T0F1,620130,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/620130,Large Projects (over 40 km),"Alberta, British Columbia, All",OH-4-2011,Jacques Whitford AXYS Ltd,Rescinded Certificates,"OC-60, OC-61, \nRO-OC-60, RO-OC-61",https://apps.cer-rec.gc.ca/REGDOCS/Item/View/620327,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2398259,"Section 2: Project Description, Section 3: Scope of Assessment and Environmental Assessment Methods, Appendix 3A: Project Inclusion List within the REAA in Alberta and British Columbia, Section 4: Atmospheric Environment (Volume 6A, Pipeline and Tank ...",1.0,"Air, All",http://www.cer-rec.gc.ca/esa-ees/nrthrngtwy/nrthrngtwy_table-2-6-estimated-barge-traffic-for-marine-termi_pt-2_pg-44_doc-num-A1T0F1.csv,44,184.0,4.85,Yes,nrthrngtwy,http://www.cer-rec.gc.ca/esa-ees/nrthrngtwy.zip,False,nrthrngtwy_table-2-6-estimated-barge-traffic-for-marine-termi_pt-2_pg-44_doc-num-A1T0F1.csv,620130_44_lattice-v_1.csv,Table 2 6 Estimated Barge Traffic for Marine Terminal Construction Construction Year Construction Quarter Trips from Vancouver to the Kitimat Terminal Trips from the Kitimat Terminal to Vancouver Total Barge Trips Year 4 1 0 0 0 Year 4 3 0 0 0 Year 4 ...,-1
8409,17543,TABLE 14 ALCES RIVER UNIT ADDITION AIR DISPERSION MODELLING RESULTS SUMMARY,Table,Application for the 2017 NGTL System Expansion,2017 NGTL System Expansion,3/31/2015,NOVA Gas Transmission Ltd.,Gas,B3-33_ESA_Appendix_4_AQ_TDR_Part1of2 - A4K2T9,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/2748828,A4K2T9,2758416,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/2758416,Large Projects (over 40 km),"Alberta, All",GH-002-2015,TERA a CHTM HILL Company,Operating,GC-126,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2758964,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2985011,Appendix 4: Air Quality and Greenhouse Gas Emissions Technical Report for the Proposed NOVA Gas Transmission Ltd. 2017 NGTL System Expansion,33.0,"Air, Technology, All",http://www.cer-rec.gc.ca/esa-ees/2017ngtl/2017ngtl_table-14-alces-river-unit-addition-air-dispersion-_pt-1_pg-26_doc-num-A4K2T9.csv,26,39.0,5.67,Yes,2017ngtl,http://www.cer-rec.gc.ca/esa-ees/2017ngtl.zip,False,2017ngtl_table-14-alces-river-unit-addition-air-dispersion-_pt-1_pg-26_doc-num-A4K2T9.csv,2758416_26_lattice-v_1.csv,TABLE 14 ALCES RIVER UNIT ADDITION AIR DISPERSION MODELLING RESULTS SUMMARY Parameters Project Concentrations Cumulative Concentrations 1 Background Concentrations Overall Concentrations 2 AAAQO g m 3 g m 3 g m 3 g m 3 g m 3 NO 2 1 hour 98 95 130 64 1...,-1


In [37]:
df_for_labeling.head(2)

,Index,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,bad_csv,CSV Filename,csvFileName,text,label
8889,18023,TABLE 8A.2 REASONABLY FORSEEABLE DEVELOPMENTS IDENTIFIED WITHIN THE SPATIAL BOUNDARIES OF THE BEAR CANYON SECTION,Table,Application for the 2017 NGTL System Expansion,2017 NGTL System Expansion,3/31/2015,NOVA Gas Transmission Ltd.,Gas,B2-15_ESA_Main_Sec_8_Part1of2 - A4K2S0,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/2748481,A4K2S0,2758825,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/2758825,Large Projects (over 40 km),"Alberta, All",GH-002-2015,TERA a CHTM HILL Company,Operating,GC-126,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2758964,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2985011,Section 8: Cumulative Effects Assessment,15.0,"Other, All",http://www.cer-rec.gc.ca/esa-ees/2017ngtl/2017ngtl_table-8a-2-reasonably-forseeable-developments-iden_pt-10_pg-359_doc-num-A4K2S0.csv,359,472.0,6.50,Yes,2017ngtl,http://www.cer-rec.gc.ca/esa-ees/2017ngtl.zip,False,2017ngtl_table-8a-2-reasonably-forseeable-developments-iden_pt-10_pg-359_doc-num-A4K2S0.csv,2758825_359_lattice-v_1.csv,TABLE 8A 2 REASONABLY FORSEEABLE DEVELOPMENTS IDENTIFIED WITHIN THE SPATIAL BOUNDARIES OF THE BEAR CANYON SECTION Map ID 1 Title Proponent Description Construction Schedule 2 Assessment Type Quantitative or Qualitative Physical Environment and Soils L...,-1
27753,36887,TABLE 1 RESOURCE-SPECIFIC MITIGATION TABLE,Table,Application for the Northwest Mainline Expansion,Northwest Mainline,4/29/2011,NOVA Gas Transmission Ltd.,Gas,B3-38 - 7212_App8_EPP - A1Y9J1,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/684358,A1Y9J1,684515,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/684515,Large Projects (over 40 km),"Alberta, British Columbia, All",GH-2-2011,TERA Environmental Consultants,Operating,GC-119,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/685859,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/793577,Appendix 8: Environmental Protection Plan,23.0,"Environment Protection Plan, All",http://www.cer-rec.gc.ca/esa-ees/nrthwstmnln/nrthwstmnln_table-1-resource-specific-mitigation-table_pt-3_pg-22_doc-num-A1Y9J1.csv,22,194.0,4.43,Yes,nrthwstmnln,http://www.cer-rec.gc.ca/esa-ees/nrthwstmnln.zip,False,nrthwstmnln_table-1-resource-specific-mitigation-table_pt-3_pg-22_doc-num-A1Y9J1.csv,684515_22_lattice-v_1.csv,TABLE 1 RESOURCE SPECIFIC MITIGATION TABLE Location Issues Timing Windows and or Regulatory Guidelines Mitigation Comments VEGETATION General All Three Sections of Pipeline Rare Plant Species and Rare Ecological Communities N A The locations of rare p...,-1


In [38]:
df_for_labeling.to_csv(str(ROOT_PATH / 'data/interim/Intermediate_Index_Files/esa_index_for_labeling.csv'), index=False, encoding='utf-8')